In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
import findspark
findspark.init()
from pyspark import SparkContext

In [5]:
sc=SparkContext(appName="mysparkcorejoinsapp",master="local[*]").getOrCreate()

In [6]:
sc


<SparkContext master=local[*] appName=mysparkcorejoinsapp>

In [9]:
custs=sc.textFile("/content/custs_without_header.csv")

In [10]:
custsmap=custs.map(lambda str:str.split(","))

In [11]:
custsmap.first()

['4000001', 'Kristina', 'Chung', '55', 'Pilot']

In [12]:
custsmaprdd=custsmap.map(lambda x:(x[0],x[1]))

In [13]:
custsmaprddreduce=custsmaprdd.reduceByKey(lambda x,y:(x+y))

In [14]:
custsmaprddreduce.collect()

[('4000004', 'Gretchen'),
 ('4000007', 'Elsie'),
 ('4000008', 'Hazel'),
 ('4000012', 'Sandy'),
 ('4000013', 'Marion'),
 ('4000014', 'Beth'),
 ('4000020', 'Crystal'),
 ('4000024', 'Franklin'),
 ('4000026', 'Marian'),
 ('4000027', 'Marcia'),
 ('4000029', 'Wayne'),
 ('4000030', 'Stephanie'),
 ('4000031', 'Neal'),
 ('4000032', 'Gretchen'),
 ('4000033', 'Tim'),
 ('4000036', 'Priscilla'),
 ('4000037', 'Elsie'),
 ('4000041', 'Donald'),
 ('4000046', 'Louis'),
 ('4000047', 'Christina'),
 ('4000048', 'Darlene'),
 ('4000051', 'Frederick'),
 ('4000054', 'Judith'),
 ('4000059', 'Pat'),
 ('4000061', 'Jessica'),
 ('4000062', 'Evan'),
 ('4000063', 'Melinda'),
 ('4000068', 'Allan'),
 ('4000071', 'Natalie'),
 ('4000072', 'Caroline'),
 ('4000074', 'Kyle'),
 ('4000075', 'Calvin'),
 ('4000078', 'Sara'),
 ('4000079', 'Stacy'),
 ('4000080', 'Gladys'),
 ('4000081', 'Mike'),
 ('4000083', 'Faye'),
 ('4000085', 'Leon'),
 ('4000089', 'Sherri'),
 ('4000090', 'Patsy'),
 ('4000093', 'Curtis'),
 ('4000094', 'Dana'),


In [15]:
tnx=sc.textFile("/content/txns_without_header.csv")

In [18]:
tnxrdd=tnx.map(lambda str:str.split(","))
tnxrdd.first()

['00000000',
 '06-26-2011',
 '4007024',
 '040.33',
 'Exercise & Fitness',
 'Cardio Machine Accessories',
 'Clarksville',
 'Tennessee',
 'credit']

In [27]:
tnxrddmap=tnxrdd.map(lambda x:(x[2],(float(x[3]))))
tnxrddmap.first()

('4007024', 40.33)

In [28]:
tnxrddmapreduce=tnxrddmap.reduceByKey(lambda x,y:(x+y))

In [30]:
tnxrddmapreduce.collect()

[('4009775', 541.6400000000001),
 ('4002199', 386.71),
 ('4002613', 415.53000000000003),
 ('4007361', 557.97),
 ('4004646', 1202.78),
 ('4002473', 955.35),
 ('4003179', 904.96),
 ('4009135', 548.75),
 ('4001364', 532.87),
 ('4003144', 201.44),
 ('4006131', 834.02),
 ('4005691', 626.38),
 ('4009693', 297.03),
 ('4002130', 390.42999999999995),
 ('4006967', 470.7),
 ('4005737', 596.98),
 ('4004237', 652.0200000000001),
 ('4005646', 306.11),
 ('4005580', 534.44),
 ('4008289', 364.13),
 ('4003091', 368.79),
 ('4007357', 462.02),
 ('4008744', 185.07),
 ('4004874', 351.09000000000003),
 ('4009680', 789.28),
 ('4008455', 786.07),
 ('4006293', 433.65000000000003),
 ('4002526', 702.55),
 ('4006736', 647.76),
 ('4005403', 525.49),
 ('4006291', 469.75),
 ('4001305', 945.3799999999999),
 ('4008653', 409.09),
 ('4006262', 245.79),
 ('4003860', 260.21000000000004),
 ('4000458', 392.67),
 ('4002200', 915.75),
 ('4005751', 1192.99),
 ('4005625', 918.4699999999999),
 ('4007259', 302.44),
 ('4008024', 32

In [33]:
joined=custsmaprddreduce.join(tnxrddmapreduce)

In [34]:
joined.take(5)

[('4000004', ('Gretchen', 672.54)),
 ('4000007', ('Elsie', 719.6600000000001)),
 ('4000012', ('Sandy', 281.07)),
 ('4000013', ('Marion', 307.63)),
 ('4000014', ('Beth', 596.1700000000001))]

In [35]:
joinedrdd=joined.map(lambda x:(x[0],x[1][0],x[1][1]))

In [36]:
joinedrdd.take(10)

[('4000004', 'Gretchen', 672.54),
 ('4000007', 'Elsie', 719.6600000000001),
 ('4000012', 'Sandy', 281.07),
 ('4000013', 'Marion', 307.63),
 ('4000014', 'Beth', 596.1700000000001),
 ('4000020', 'Crystal', 576.5699999999999),
 ('4000024', 'Franklin', 367.59000000000003),
 ('4000026', 'Marian', 299.41),
 ('4000027', 'Marcia', 248.58),
 ('4000029', 'Wayne', 487.59)]